<a href="https://colab.research.google.com/github/Strojove-uceni/23206-final-acoustic-emission-classification/blob/main/AE_classificatio_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Imports and Helper Functions
%%capture
!pip install lightning
!pip3 install wandb